In [2]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing
import pathlib
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import os.path 
import re
from datetime import datetime
from konlpy.tag import Komoran
import json


#### !필독!

1. 학습을 시켰을때와 대체로 비슷한 구조의 파일입니다. 다만 학습시키는 과정과, 기타 전처리 과정들이 빠져있을뿐..
2. 주의할 점은 학습시켰을때 썼던 파일 순서대로 dataset 리스트에 넣어주세요. 안그러면 convert_to_originalSentense() 썼을때 key error오류날껍니다. 또한 학습 시킨 갯수(trainCount)도 동일하게 해주셔야 합니다
 - ex) ['1.xlxs', '2.xlsx', '3.xlsx'] -> 학습 시켰을 때 / ['1.xlxs', '2.xlsx', '3.xlsx'] -> 현재 파일에서 예측 시킬때
3. RestAPI는 플라스크에서 제공하는 CORS를 사용했습니다. 안그러면 크롬이나 기타 브라우저에서 오류납니다. pip로 깔아주세요
4. 만약 하다가 데이터셋이 꼬였다 싶으면... datatSetClean() 함수를 통해서 데이터셋 클리어 해주시고 setDataSet() 함수로 다시 설정해주면 될껍니다
5. 주소는 바꾸되 Post로 안하면 사용자의 돌발행동으로 인해서 오류가 날수도 있습니다. (질문이 1000자정도 넘어갈때)
6. 플라스크는 테스트 안해봤습니다!! 입맛에 따라 바꾸세요!! 아마 안될리가 없긴 합니다만

In [3]:
class chatbot:
    PAD = "<PADDING>"   # 패딩
    STA = "<START>"     # 시작
    END = "<END>"       # 끝
    OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)
    # 태그 인덱스
    PAD_INDEX = 0
    STA_INDEX = 1
    END_INDEX = 2
    OOV_INDEX = 3
    # 데이터 타입
    ENCODER_INPUT  = 0
    DECODER_INPUT  = 1
    DECODER_TARGET = 2
    # 한 문장에서 단어 시퀀스의 최대 개수
    max_sequences = 100
    RE_FILTER = re.compile("[.,!?\"':;~()]")
    dataSet = pd.DataFrame([])
    question = []
    answer = []
    word_to_index = None
    index_to_word = None
    x_encoder = None
    x_decoder = None
    y_decoder = None
    model = None
    encoder_model = None
    decoder_model = None
    originalDict = {}
    
    def setDataSet(self, excelFile):
        # 데이터 셋 집어넣을 때 반드시 컬럼명을 Q, A로 지정해주셔야 합니다.
        # 또한 엑셀파일 전용입니다.
        self.dataSet = self.dataSet.append(pd.read_excel(excelFile,encoding='utf-8'))
        self.question, self.answer = list(self.dataSet['Q']), list(self.dataSet['A'])
    def dataSetClean(self):
        self.dataSet = pd.DataFrame([])
    def setMaxSequences(self,maxSequences):
        self.max_sequences = maxSequences
    def setEmbedding_dim(self, embeddingDim):
        self.embedding_dim = embeddingDim
    # 형태소분석 함수
    def pos_tag(self, sentences):
        # KoNLPy 형태소분석기 설정
        tagger = Komoran()
        # 문장 품사 변수 초기화
        sentences_pos = []
        # 모든 문장 반복
        for sentence in sentences:
            # 특수기호 제거
            sentence = re.sub(self.RE_FILTER, "", sentence)
            # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
            sentence = " ".join(tagger.morphs(sentence))
            sentences_pos.append(sentence)
        return sentences_pos
    def startPosTag(self):
        self.question = self.pos_tag(self.question)
        self.answer = self.pos_tag(self.answer)
       
    def getSetUpfile(self, word_to_index, index_to_word, x_encoder, x_decoder, y_decoder, model, encoder_model, decoder_model):
        file = pathlib.Path(word_to_index)
        file_text = file.read_text(encoding='utf-8')
        self.word_to_index = json.loads(file_text)
        
        file = pathlib.Path(index_to_word)
        file_text = file.read_text(encoding='utf-8')
        self.index_to_word = json.loads(file_text)
        self.index_to_word = {index:word for index,word in enumerate(self.index_to_word.values())}
        
        self.x_encoder = pd.read_csv(x_encoder)
        del(self.x_encoder['Unnamed: 0'])
        self.x_encoder = np.array(self.x_encoder)
        
        self.x_decoder = pd.read_csv(x_decoder)
        del(self.x_decoder['Unnamed: 0'])
        self.x_decoder = np.array(self.x_decoder)
        
        self.y_decoder = pd.read_csv(y_decoder)
        del(self.y_decoder['Unnamed: 0'])
        self.y_decoder = np.array(self.y_decoder)
        
        self.model = load_model(model) 
        self.encoder_model = load_model(encoder_model)
        self.decoder_model = load_model(decoder_model)
        
    def convert_text_to_index(self, sentences, vocabulary, type): 
        sentences_index = []
        # 모든 문장에 대해서 반복
        for sentence in sentences:
            sentence_index = []
            # 디코더 입력일 경우 맨 앞에 START 태그 추가
            if type == self.DECODER_INPUT:
                sentence_index.extend([vocabulary[self.STA]])

            # 문장의 단어들을 띄어쓰기로 분리
            for word in sentence.split():
                if vocabulary.get(word) is not None:
                    # 사전에 있는 단어면 해당 인덱스를 추가
                    sentence_index.extend([vocabulary[word]])
                else:
                    # 사전에 없는 단어면 OOV 인덱스를 추가
                    sentence_index.extend([vocabulary[self.OOV]])

            # 최대 길이 검사
            if type == self.DECODER_TARGET:
                # 디코더 목표일 경우 맨 뒤에 END 태그 추가
                if len(sentence_index) >= self.max_sequences:
                    sentence_index = sentence_index[:self.max_sequences-1] + [vocabulary[self.END]]
                else:
                    sentence_index += [vocabulary[self.END]]
            else:
                if len(sentence_index) > self.max_sequences:
                    sentence_index = sentence_index[:self.max_sequences]

            # 최대 길이에 없는 공간은 패딩 인덱스로 채움
            sentence_index += (self.max_sequences - len(sentence_index)) * [vocabulary[self.PAD]]

            # 문장의 인덱스 배열을 추가
            sentences_index.append(sentence_index)

        return np.asarray(sentences_index)
   
    
    def convert_index_to_text(self, indexs, vocabulary): 
    
        sentence = ''

        # 모든 문장에 대해서 반복
        for index in indexs:
            if index == self.END_INDEX:
                # 종료 인덱스면 중지
                break;
            if vocabulary.get(index) is not None:
                # 사전에 있는 인덱스면 해당 단어를 추가
                sentence += vocabulary[index]
            else:
                # 사전에 없는 인덱스면 OOV 단어를 추가
                sentence.extend([vocabulary[self.OOV_INDEX]])

            # 빈칸 추가
            sentence += ' '

        return sentence
    def make_predict_input(self, sentence):

        sentences = []
        sentences.append(sentence)
        sentences = self.pos_tag(sentences)
        input_seq = self.convert_text_to_index(sentences, self.word_to_index, self.ENCODER_INPUT)

        return input_seq
    # 텍스트 생성
    def generate_text(self, input_seq):

        # 입력을 인코더에 넣어 마지막 상태 구함
        states = self.encoder_model.predict(input_seq)

        # 목표 시퀀스 초기화
        target_seq = np.zeros((1, 1))

        # 목표 시퀀스의 첫 번째에 <START> 태그 추가
        target_seq[0, 0] = self.STA_INDEX

        # 인덱스 초기화
        indexs = []

        # 디코더 타임 스텝 반복
        while 1:
            # 디코더로 현재 타임 스텝 출력 구함
            # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
            decoder_outputs, state_h, state_c = self.decoder_model.predict(
                                                    [target_seq] + states)

            # 결과의 원핫인코딩 형식을 인덱스로 변환
            index = np.argmax(decoder_outputs[0, 0, :])
            indexs.append(index)

            # 종료 검사
            if index == self.END_INDEX or len(indexs) >= self.max_sequences:
                break

            # 목표 시퀀스를 바로 이전의 출력으로 설정
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = index

            # 디코더의 이전 상태를 다음 디코더 예측에 사용
            states = [state_h, state_c]

        # 인덱스를 문장으로 변환
        sentence = self.convert_index_to_text(indexs, self.index_to_word)

        return sentence
    
    def makePredictSentense(self,targetSentense):
        return self.generate_text(self.make_predict_input(targetSentense))
    
    def makeOriginalSentenseDict(self, targetSentenseList):
        for sentense in targetSentenseList :
            self.originalDict[self.makePredictSentense(sentense)] = sentense
            
    def convert_to_originalSentense(self, predictSentense):
        sentense = self.makePredictSentense(predictSentense)
        return self.originalDict[sentense]
    
    #
    def AutoStart(self, dataSet,trainCount, word_to_index, index_to_word, x_encoder, x_decoder, y_decoder, model, encoder_model, decoder_model):
        for i in dataSet:
            self.setDataSet(i)
        self.startPosTag()
        self.getSetUpfile(word_to_index, index_to_word, x_encoder, x_decoder, y_decoder, model, encoder_model, decoder_model)
        self.makeOriginalSentenseDict(self.dataSet['A'][:trainCount])

In [4]:
cb = chatbot()

In [5]:
dataset = ['dataset/chatbot/예제.xlsx']

In [6]:
# 요건 예제이니 바꾸시면 됩니다.
cb.AutoStart(dataset, trainCount=10, index_to_word = './json/20200815_index_to_word.json',
            word_to_index = './json/20200815_word_to_index.json', x_encoder = './EnDecoder/20200815_x_encoder.csv',
            x_decoder = './EnDecoder/20200815_x_decoder.csv', y_decoder = './EnDecoder/20200815_y_decoder.csv', 
            model = './model/20200815_model.h5', encoder_model = './model/20200815_encoder_model.h5',
            decoder_model = './model/20200815_decoder_model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/chatbot/예제.xlsx'

In [37]:
cb.makePredictSentense('코로나란 무엇인가요?')

'코로나바이러스 감염증 - 19 코로나 19 는 과거 에 발견 되 지 않 았 던 새롭 ㄴ 코로나바이러스 이 ㄴ SARS - CoV -2 에 의하 아 발생 하 는 호흡기 감염병 입 니다 이 바이러스 에 감염 되 면 무증상 부터 중증 에 이르 기 까지 다양 하 ㄴ 임상 증상 이 나타나 ㄹ 수 있 습니다 이 새롭 ㄴ 바이러스 와 질병 은 2019 년 12월 중국 우 한 에서 처음 보고 되 었 고 현재 전 세계 에 확산 되 었 습니다 '

In [49]:
from flask import Flask, request, jsonify
app = Flask(__name__)
from flask_cors import CORS
from datetime import datetime
@app.route('/dto/',methods = ['POST'])
def chatbotServer():
    Q = request.form['Q']
    print(Q)
    A = cb.convert_to_originalSentense(Q)
    return jsonify({'Q' : Q, 'A' : A, 'datetime' : datetime.today().strftime("%Y%m%d%H%M%S")})
    
if __name__ == "__main__":
    CORS(app)
    app.run(port=5000 , threaded = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
